In [1]:
import pymolviz as pmv
import numpy as np

## GridData
Volumetric Displays are all based on the _GridData_ class. As the name implies it stores data on a grid. The grid can either be specified by its _positions_ or by an _origin_, _step_sizes_ and _step_counts_, each being a 3-dimensional vector. At least currently non-orthogonal Grids are not supported. _GridData_ is a _Displayable_, and thus can be loaded into PyMOL on its own.

As a simple example we create a 1 by 1 cube at \[0, 0, 0\] and assign values as the  hamming distance from origin.

In [2]:
grid = np.array([[0, 0, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 1, 1],
        [1, 0, 0],
        [1, 0, 1],
        [1, 1, 0],
        [1, 1, 1]])

values = np.sum(grid, axis=1)
grid_data = pmv.GridData(values, grid, name = "grid_data")
grid_data.write("out/basic_grid_data.py")

In [3]:
points = grid_data.to_points(name = "grid_data_points", radius = 0.1)
points.write("out/basic_grid_data_points.py")

## Reading external volumetric data
PyMOLViz currently supports reading xyz files containing volumetric data created by turbomole, as well as mtz files using [_gemmi_](https://gemmi.readthedocs.io/en/latest/).

### Reading xyz files
One example was already in the _Points_ chapter.

The _from_xyz_ function has the argument _in_bohr_ which defaults to True. This signifies that the data from the xyz file is in Bohr (as is the default for turbomole outputs), but we want to see it in Angström. If you want to leave the data in its original form you need to pass `in_bohr = False`. 

In [4]:
data = pmv.GridData.from_xyz("../data/td.xyz", name = "td")
cmap = pmv.ColorMap([0, 0.5])
points = data.to_points(name = "td_points", radius = 0.1, render_as="Dots", colormap = cmap)
pmv.Script([data, points]).write("out/td_points.py")

### Reading mtz files
Reading mtz files is more complicated, since they usually hold a lot of information and you need to pick what information you would like to display. If you want more than a quick way to display the information you need to take a deeper look into the documentation, e.g. for the [MTZ Format](https://www.ccp4.ac.uk/html/mtzformat.html) or how to handle them with [gemmi](https://gemmi.readthedocs.io/en/latest/hkl.html).

PyMOLViz allows to specify the structure factor column and the phase column from the mtz file to compute the grid data. additionally you can specify the sample rate and the area in which to display the data via the _min_pos_ and _max_pos_ keywords.

For standard files you want the following structure factors and phases (see https://www.globalphasing.com/buster/wiki/index.cgi?MTZcolumns, https://www.ccp4.ac.uk/html/refmac5/keywords/xray-general.html):
* 2mFo-dFc: Factor: "FWT", Phase: "PHWT"
* mFo-dFc: Factor: "DELFWT", Phase: "PHDELWT"
* Fc: Factor: "FC", Phase: "PHIC"
* Fo: Factor: "FP", Phase: usually N/A

In [5]:
from rdkit import Chem
prot = Chem.MolFromPDBFile('../data/4de3.pdb')
positions = prot.GetConformer().GetPositions()
buffer = 5
min_pos = np.min(positions, axis = 0) - buffer
max_pos = np.max(positions, axis = 0) + buffer
data = pmv.GridData.from_mtz('../data/4de3_phases.mtz', min_pos = min_pos, max_pos = max_pos, step_sizes = [1, 1, 1], name = 'mtz_4de3')
data.write("out/mtz_4de3.py")